In [1]:
%cd "~/pln-brca-xp/notebooks"
%load_ext autoreload
%autoreload 2

/home/xabush/pln-brca-xp/notebooks


In [2]:
import pickle

from goatools.associations import dnld_assc
from goatools.obo_parser import GODag
from goatools.semantic import *

from convert_symbol_to_entrez import read_symbol2geneid
from utils import *
from utils_emb import *

In [3]:
feats_diff = load_features("/var/www/datasets/filteredGenes4studies.txt")
non_embedded = load_features("datasets/non_embedded_genes_list.txt")
feats_diff_filtered = list(set(feats_diff) - set(non_embedded))
len(feats_diff_filtered)

4382

In [4]:
godag = GODag("go-basic.obo")
associations = dnld_assc("datasets/goa_human.gaf", godag, namespace="all")
termcounts = TermCounts(godag, associations)

go-basic.obo: fmt(1.2) rel(2021-02-01) 47,291 GO Terms
HMS:0:00:11.048003 479,438 annotations READ: datasets/goa_human.gaf 
19783 IDs in loaded association branch, all


In [5]:
pln_go_df = pd.read_csv("datasets/pln_goa_comp/goa_pln_diff_2.csv")
goa_df = pd.read_csv("datasets/pln_goa_comp/goa_pln_diff_1.csv")
pln_lst = pln_go_df[pln_go_df["NS"] == "BP"]["# GO"].to_list()
goa_lst = goa_df[goa_df["NS"] == "BP"]["# GO"].to_list()

In [6]:
with open("datasets/pubmed_gene_ann.pickle", "rb") as fp:
    pubmed_gene_ann = pickle.load(fp)

In [112]:
len(pubmed_gene_ann)

1265

In [7]:
pubmed_genes = []

for k, v in pubmed_gene_ann.items():
    pubmed_genes.extend(v)

pubmed_genes = set(pubmed_genes)
len(pubmed_genes)

1617

In [8]:
sym2geneid = read_symbol2geneid("datasets/Homo_sapiens.gene_info")

  126,696 READ: datasets/Homo_sapiens.gene_info


In [9]:
pop_ids = convert_symbol_to_geneid(feats_diff_filtered ,sym2geneid)
pubmed_intr = list(pubmed_genes & set(pop_ids))
len(pubmed_intr)

530

In [42]:
pubmed_go_df = run_gene_enrich(pubmed_intr, pop_ids, "datasets/pln_goa_comp/list_4_pubmed" ,sym2geneid, convert_study=False, convert_pop=False)
display(pubmed_go_df)

datasets/go-basic.obo: fmt(1.2) rel(2021-02-01) 47,291 GO Terms; optional_attrs(relationship)
HMS:0:00:05.164596 335,863 annotations, 20,672 genes, 18,441 GOs, 1 taxids READ: gene2go 
Study: 530 vs. Population 4382


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 97%  4,250 of  4,382 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 99%  4,328 of  4,382 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
 98%  4,296 of  4,382 population items found in association

Run BP Gene Ontology Analysis: current study set of 530 IDs ... 99%    527 of    530 study items found in association
100%    530 of    530 study items found in population(4382)
Calcu

,# GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,p_fdr_bh,study_items
0,GO:0010941,BP,e,regulation of cell death,189/530,608/4382,5.337717e-43,4,189,6.220576e-39,6.063745e-39,6.220576e-39,6.220576e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
1,GO:0043067,BP,e,regulation of programmed cell death,180/530,562/4382,1.082218e-42,5,180,1.261217e-38,1.229420e-38,1.261109e-38,6.306087e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
2,GO:0009893,BP,e,positive regulation of metabolic process,290/530,1242/4382,2.430488e-42,4,290,2.832491e-38,2.761079e-38,2.832004e-38,9.441635e-39,"47, 142, 183, 196, 199, 207, 301, 367, 374, 38..."
3,GO:0048518,BP,e,positive regulation of biological process,378/530,1923/4382,5.160696e-42,3,378,6.014275e-38,5.862646e-38,6.012727e-38,1.503569e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
4,GO:0048522,BP,e,positive regulation of cellular process,360/530,1780/4382,1.037532e-41,4,360,1.209140e-37,1.178656e-37,1.208725e-37,2.418280e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,GO:0003954,MF,p,NADH dehydrogenase activity,0/530,29/4382,4.196005e-02,4,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN
3200,GO:0008137,MF,p,NADH dehydrogenase (ubiquinone) activity,0/530,29/4382,4.196005e-02,7,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN
3201,GO:0004553,MF,p,"hydrolase activity, hydrolyzing O-glycosyl com...",0/530,29/4382,4.196005e-02,4,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN
3202,GO:0050136,MF,p,NADH dehydrogenase (quinone) activity,0/530,29/4382,4.196005e-02,6,0,1.000000e+00,1.000000e+00,1.000000e+00,4.001862e-01,NaN


In [43]:
pubmed_go_bp_df = pubmed_go_df[pubmed_go_df["NS"] == "BP"]
pubmed_go_bp = pubmed_go_bp_df["# GO"].to_list()
len(pubmed_go_bp)

2687

In [15]:
overlap_pln, _, _ = find_overlap_go(pln_lst, pubmed_go_bp)

Num overlap: 184
Num found in list 1, not in list 2:221
Num found in list 2, not in list 1: 2503


In [16]:
overlap_goa, _, _ = find_overlap_go(goa_lst, pubmed_go_bp)

Num overlap: 98
Num found in list 1, not in list 2:95
Num found in list 2, not in list 1: 2589


In [17]:
def calc_overlap(s1, s2):
    intr = len(s1 & s2)
    return intr / (len(s1) + len(s2) - intr)

In [20]:
pln_pubmed_go_df = pubmed_go_df[pubmed_go_df["# GO"].isin(overlap_pln)]
pln_pubmed_go_df.set_index("# GO", inplace=True)
display(pln_pubmed_go_df)

,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,p_fdr_bh,study_items
# GO,,,,,,,,,,,,,
GO:0010941,BP,e,regulation of cell death,189/530,608/4382,5.337717e-43,4,189,6.220576e-39,6.063745e-39,6.220576e-39,6.220576e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
GO:0043067,BP,e,regulation of programmed cell death,180/530,562/4382,1.082218e-42,5,180,1.261217e-38,1.229420e-38,1.261109e-38,6.306087e-39,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
GO:0048518,BP,e,positive regulation of biological process,378/530,1923/4382,5.160696e-42,3,378,6.014275e-38,5.862646e-38,6.012727e-38,1.503569e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
GO:0048522,BP,e,positive regulation of cellular process,360/530,1780/4382,1.037532e-41,4,360,1.209140e-37,1.178656e-37,1.208725e-37,2.418280e-38,"47, 142, 183, 196, 199, 207, 220, 292, 301, 33..."
GO:0042981,BP,e,regulation of apoptotic process,175/530,547/4382,3.077777e-41,6,175,3.586841e-37,3.496411e-37,3.585302e-37,5.978068e-38,"142, 183, 207, 220, 292, 301, 308, 332, 367, 3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0006811,BP,p,ion transport,89/530,890/4382,3.303172e-02,4,89,1.000000e+00,1.000000e+00,1.000000e+00,1.652862e-01,"207, 292, 301, 347, 360, 847, 967, 972, 1020, ..."
GO:0072657,BP,p,protein localization to membrane,15/530,203/4382,3.570039e-02,5,15,1.000000e+00,1.000000e+00,1.000000e+00,1.751800e-01,"999, 1020, 2316, 3551, 5590, 5906, 6386, 7074,..."
GO:0006413,BP,p,translational initiation,4/530,82/4382,3.989929e-02,3,4,1.000000e+00,1.000000e+00,1.000000e+00,1.836771e-01,"1968, 1977, 7311, 8894"


In [62]:
pubmed_id_gos_pln = {}

for pubmed_id in pubmed_gene_ann:
    pubmed_id_gos_pln[pubmed_id] = []
    for go in overlap_pln:
        study_item_ls = pln_pubmed_go_df.loc[go]["study_items"]
        #check if we got a nan value
        if isinstance(study_item_ls, float): continue
        study_item_ls = study_item_ls.split(",")
        study_items = [int(x.strip()) for x in study_item_ls]
        overlap_ratio = calc_overlap(pubmed_gene_ann[pubmed_id], set(study_items))
        # if len(pubmed_gene_ann[pubmed_id] & set(study_items)) > 0:
        #     pubmed_id_gos_pln[pubmed_id].append(go)
        if overlap_ratio > 0.1:
            pubmed_id_gos_pln[pubmed_id].append(go)
    if len(pubmed_id_gos_pln[pubmed_id]) == 0:
        del pubmed_id_gos_pln[pubmed_id]

len(pubmed_id_gos_pln)

277

In [25]:
goa_pubmed_go_df = pubmed_go_df[pubmed_go_df["# GO"].isin(overlap_goa)]
goa_pubmed_go_df.set_index("# GO", inplace=True)
display(goa_pubmed_go_df)

,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,p_fdr_bh,study_items
# GO,,,,,,,,,,,,,
GO:0048523,BP,e,negative regulation of cellular process,307/530,1524/4382,4.015752e-31,4,307,4.679957e-27,4.561968e-27,4.671122e-27,2.034764e-28,"142, 183, 196, 199, 207, 292, 301, 308, 332, 3..."
GO:0048519,BP,e,negative regulation of biological process,325/530,1700/4382,6.059478e-29,3,325,7.061716e-25,6.883679e-25,7.044750e-25,2.435075e-26,"142, 183, 196, 199, 207, 216, 292, 301, 308, 3..."
GO:0048869,BP,e,cellular developmental process,186/530,737/4382,3.872469e-28,2,186,4.512975e-24,4.399196e-24,4.499422e-24,1.253604e-25,"142, 207, 230, 301, 367, 387, 427, 467, 468, 4..."
GO:0031399,BP,e,regulation of protein modification process,153/530,557/4382,8.297100e-27,7,153,9.669440e-23,9.425659e-23,9.632933e-23,2.148765e-24,"183, 207, 374, 387, 472, 595, 604, 648, 836, 8..."
GO:0044093,BP,e,positive regulation of molecular function,159/530,601/4382,5.961731e-26,3,159,6.947801e-22,6.772636e-22,6.917992e-22,1.339536e-23,"142, 183, 207, 216, 367, 374, 387, 389, 396, 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0010838,BP,e,positive regulation of keratinocyte proliferation,2/530,3/4382,4.029234e-02,7,2,1.000000e+00,1.000000e+00,1.000000e+00,1.836771e-01,"374, 4853"
GO:1905064,BP,e,negative regulation of vascular associated smo...,2/530,3/4382,4.029234e-02,8,2,1.000000e+00,1.000000e+00,1.000000e+00,1.836771e-01,"1786, 4775"
GO:2000615,BP,e,regulation of histone H3-K9 acetylation,2/530,3/4382,4.029234e-02,11,2,1.000000e+00,1.000000e+00,1.000000e+00,1.836771e-01,"2625, 4089"


In [63]:
pubmed_id_gos_goa = {}

for pubmed_id in pubmed_gene_ann:
    pubmed_id_gos_goa[pubmed_id] = []
    for go in overlap_goa:
        study_item_ls = goa_pubmed_go_df.loc[go]["study_items"]
        #check if we got a nan value
        if isinstance(study_item_ls, float): continue
        study_item_ls = study_item_ls.split(",")
        study_items = [int(x.strip()) for x in study_item_ls]
        # if len(pubmed_gene_ann[pubmed_id] & set(study_items)) > 0:
        #     pubmed_id_gos_goa[pubmed_id].append(go)
        overlap_ratio = calc_overlap(pubmed_gene_ann[pubmed_id], set(study_items))
        if overlap_ratio > 0.1:
            pubmed_id_gos_goa[pubmed_id].append(go)

    if len(pubmed_id_gos_goa[pubmed_id]) == 0:
        del pubmed_id_gos_goa[pubmed_id]

len(pubmed_id_gos_goa)


245

In [64]:
pubmed_id_pln = [k.strip() for k in pubmed_id_gos_pln]
pubmed_id_goa = [k.strip() for k in pubmed_id_gos_goa]
print(len(pubmed_id_pln))
print(len(pubmed_id_goa))

277
245


In [65]:
pln_goa_pubmed_diff = list(set(pubmed_id_pln) - set(pubmed_id_goa))
print(len(pln_goa_pubmed_diff))
print(pln_goa_pubmed_diff)
goa_pln_pubmed_diff = list(set(pubmed_id_goa) - set(pubmed_id_pln))
print(len(goa_pln_pubmed_diff))
print(goa_pln_pubmed_diff)

132
['33800000', '31372497', '33721131', '31306849', '31821370', '33968066', '31904863', '33836774', '33859496', '31476594', '32417506', '31876619', '31277414', '30547831', '33777781', '33816619', '32576280', '30621214', '32672418', '31394751', '33301860', '32894420', '33658859', '31417863', '32990923', '31439579', '33398005', '34016642', '33839437', '31939077', '33804148', '33449064', '31416894', '33728745', '31839155', '30830488', '32166713', '29617321', '33945630', '32415497', '33669832', '31710162', '31483418', '33857297', '30944316', '33996588', '33753724', '33840002', '32472077', '29686231', '32370060', '32758616', '33920506', '29545966', '32638964', '30996345', '32528057', '32722075', '33656060', '30573691', '33917370', '30278449', '33863872', '31220676', '32972637', '33839456', '32108307', '30597111', '30145819', '32284737', '32294525', '33335811', '33991172', '29555554', '31120927', '30431262', '32066877', '32825760', '29626519', '33734721', '33747195', '33146700', '33781662',

In [66]:
pln_goa_pubmed_diff_go = {k: v for k, v in pubmed_id_gos_pln.items() if k in pln_goa_pubmed_diff}

In [96]:
from utils_emb import abstract_download
pln_pubmed_diff_abstracts, pln_invalid_pids = abstract_download(pubmed_id_pln)
goa_pubmed_diff_abstracts, goa_invalid_pids = abstract_download(pubmed_id_goa)


In [97]:
pln_invalid_pids

['33907720', '33714027', '32644137', '32524658', '32027353']

In [101]:
pln_goa_pubmed_diff_dict = {"pubmed_id": [], "abstract": [], "go_term": []}

for k in pubmed_id_pln:
    for go in pubmed_id_gos_pln[k]:
        pln_goa_pubmed_diff_dict["pubmed_id"].append(k)
        if k in pln_invalid_pids:
            pln_goa_pubmed_diff_dict["abstract"].append("NA")
        else:
            pln_goa_pubmed_diff_dict["abstract"].append(pln_pubmed_diff_abstracts[k])

        pln_goa_pubmed_diff_dict["go_term"].append(go)

pln_goa_pubmed_diff_df = pd.DataFrame.from_dict(pln_goa_pubmed_diff_dict)
pln_goa_pubmed_diff_df

,pubmed_id,abstract,go_term
0,33996588,Uncontrolled proliferation as a result of dysr...,GO:0031570
1,33996588,Uncontrolled proliferation as a result of dysr...,GO:0000077
2,33996588,Uncontrolled proliferation as a result of dysr...,GO:0007093
3,33996588,Uncontrolled proliferation as a result of dysr...,GO:0044773
4,33997944,MicroRNAs are a group of short non-coding RNAs...,GO:0031641
...,...,...,...
553,29574992,Tamoxifen is a standard anti-hormone treatment...,GO:0006305
554,29590203,The tumor suppressor gene p53 is frequently mu...,GO:2000108
555,29617321,Exosomes are small vesicles which are produced...,GO:0031641
556,29620287,Breast cancer is one of the most common malign...,GO:0031641


In [118]:
pln_goa_pubmed_bp_df = pd.merge(pubmed_go_bp_df, pln_goa_pubmed_diff_df, left_on="# GO", right_on="go_term")
pln_goa_pubmed_bp_df.drop(["study_count", "p_bonferroni", "p_holm", "go_term"], axis=1, inplace=True)
pln_goa_pubmed_bp_df.sort_values(by="p_fdr_bh").head(20)

,# GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,p_sidak,p_fdr_bh,study_items,pubmed_id,abstract
0,GO:0048639,BP,e,positive regulation of developmental growth,23/530,47/4382,4.904488e-10,5,0.000006,2.132720e-08,"207, 891, 1385, 1432, 1627, 2066, 2335, 3479, ...",33918324,Epithelial-mesenchymal transition (EMT) is imp...
1,GO:0048639,BP,e,positive regulation of developmental growth,23/530,47/4382,4.904488e-10,5,0.000006,2.132720e-08,"207, 891, 1385, 1432, 1627, 2066, 2335, 3479, ...",32566979,IHC of single proteins or heatmap subgroups of...
2,GO:0048639,BP,e,positive regulation of developmental growth,23/530,47/4382,4.904488e-10,5,0.000006,2.132720e-08,"207, 891, 1385, 1432, 1627, 2066, 2335, 3479, ...",31867329,Breast cancer is the most prevalent tumor in w...
3,GO:0010212,BP,e,response to ionizing radiation,26/530,60/4382,1.019744e-09,4,0.000012,4.290289e-08,"142, 301, 472, 545, 595, 836, 1026, 1432, 1647...",31306849,Translocation of 78-kDa glucose-regulated prot...
4,GO:2000106,BP,e,regulation of leukocyte apoptotic process,17/530,34/4382,6.397036e-08,7,0.000727,2.059421e-06,"301, 558, 604, 972, 3091, 3575, 3958, 4067, 52...",33670518,MicroRNAs (miRNAs) are well-known regulators o...
5,GO:2000106,BP,e,regulation of leukocyte apoptotic process,17/530,34/4382,6.397036e-08,7,0.000727,2.059421e-06,"301, 558, 604, 972, 3091, 3575, 3958, 4067, 52...",30287915,Hierarchical clustering of seven PI3K/MAPK pro...
6,GO:0036294,BP,e,cellular response to decreased oxygen levels,19/530,42/4382,8.283689e-08,5,0.000941,2.609138e-06,"207, 360, 468, 890, 891, 1978, 2034, 3091, 400...",33575875,Since its initial discovery as the gene altere...
7,GO:0036294,BP,e,cellular response to decreased oxygen levels,19/530,42/4382,8.283689e-08,5,0.000941,2.609138e-06,"207, 360, 468, 890, 891, 1978, 2034, 3091, 400...",32566979,IHC of single proteins or heatmap subgroups of...
8,GO:0036294,BP,e,cellular response to decreased oxygen levels,19/530,42/4382,8.283689e-08,5,0.000941,2.609138e-06,"207, 360, 468, 890, 891, 1978, 2034, 3091, 400...",33318536,Tamoxifen resistance is emerging as a big chal...
9,GO:0031100,BP,e,animal organ regeneration,17/530,37/4382,3.067895e-07,4,0.003485,8.553410e-06,"471, 558, 595, 890, 1019, 1026, 2146, 2737, 29...",33920506,"Cancer, a disease of inappropriate cell prolif..."


In [122]:
pln_goa_pubmed_bp_df.sort_values(by="p_fdr_bh").head(100)["depth"].mean()

5.95

In [119]:
pln_goa_pubmed_bp_df["depth"].mean()

6.478494623655914

In [110]:
pln_goa_pubmed_bp_df.to_csv("datasets/pln_goa_comp/pln_goa_pubmed_bp.csv")

In [69]:
goa_pubmed_diff_go = {k: v for k, v in pubmed_id_gos_goa.items() if k in goa_pln_pubmed_diff}

In [106]:
goa_pubmed_diff_dict = {"pubmed_id": [], "abstract": [] ,"go_term": []}

for k in pubmed_id_goa:
    for go in pubmed_id_gos_goa[k]:
        goa_pubmed_diff_dict["pubmed_id"].append(k)
        if k in goa_invalid_pids:
            goa_pubmed_diff_dict["abstract"].append("NA")
        else:
            goa_pubmed_diff_dict["abstract"].append(goa_pubmed_diff_abstracts[k])
        goa_pubmed_diff_dict["go_term"].append(go)

goa_pubmed_diff_df = pd.DataFrame.from_dict(goa_pubmed_diff_dict)
goa_pubmed_diff_df

,pubmed_id,abstract,go_term
0,33999036,The study of the DNA damage response (DDR) is ...,GO:0002639
1,34000352,Approximately 70%-85% of breast cancers expres...,GO:1901184
2,34000352,Approximately 70%-85% of breast cancers expres...,GO:0038127
3,34007308,Inhibitor of growth 3 (,GO:1901184
4,34007308,Inhibitor of growth 3 (,GO:0071260
...,...,...,...
519,29512469,"Altogether, it seems that tumor cells express ...",GO:0038127
520,29574992,Tamoxifen is a standard anti-hormone treatment...,GO:0034391
521,29574992,Tamoxifen is a standard anti-hormone treatment...,GO:1905064
522,29590203,The tumor suppressor gene p53 is frequently mu...,GO:0010661


In [117]:
goa_pubmed_bp_df = pd.merge(pubmed_go_bp_df, goa_pubmed_diff_df, left_on="# GO", right_on="go_term")
goa_pubmed_bp_df.drop(["study_count", "p_bonferroni", "p_holm", "go_term"], axis=1, inplace=True)
goa_pubmed_bp_df.sort_values(by="p_fdr_bh").head(20)

,# GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,p_sidak,p_fdr_bh,study_items,pubmed_id,abstract
0,GO:0071241,BP,e,cellular response to inorganic substance,29/530,75/4382,2.784137e-09,4,0.000032,1.107383e-07,"142, 207, 468, 890, 891, 999, 1017, 1027, 1050...",33761370,The epithelial to mesenchymal transition (EMT)...
1,GO:0071241,BP,e,cellular response to inorganic substance,29/530,75/4382,2.784137e-09,4,0.000032,1.107383e-07,"142, 207, 468, 890, 891, 999, 1017, 1027, 1050...",33641663,These findings suggest that TMEM52B is a novel...
2,GO:0071241,BP,e,cellular response to inorganic substance,29/530,75/4382,2.784137e-09,4,0.000032,1.107383e-07,"142, 207, 468, 890, 891, 999, 1017, 1027, 1050...",33582643,"Tetrachlorobisphenol A (TCBPA), a chlorinated ..."
3,GO:0071241,BP,e,cellular response to inorganic substance,29/530,75/4382,2.784137e-09,4,0.000032,1.107383e-07,"142, 207, 468, 890, 891, 999, 1017, 1027, 1050...",33572896,The search for novel anti-cancer compounds whi...
11,GO:0071248,BP,e,cellular response to metal ion,26/530,66/4382,1.187717e-08,5,0.000135,4.408171e-07,"142, 207, 468, 891, 999, 1027, 1050, 1147, 138...",30832689,Our results uncover the mechanism through whic...
9,GO:0071248,BP,e,cellular response to metal ion,26/530,66/4382,1.187717e-08,5,0.000135,4.408171e-07,"142, 207, 468, 891, 999, 1027, 1050, 1147, 138...",32266132,It is now well-established that sphingosine ki...
8,GO:0071248,BP,e,cellular response to metal ion,26/530,66/4382,1.187717e-08,5,0.000135,4.408171e-07,"142, 207, 468, 891, 999, 1027, 1050, 1147, 138...",33572896,The search for novel anti-cancer compounds whi...
10,GO:0071248,BP,e,cellular response to metal ion,26/530,66/4382,1.187717e-08,5,0.000135,4.408171e-07,"142, 207, 468, 891, 999, 1027, 1050, 1147, 138...",31867329,Breast cancer is the most prevalent tumor in w...
6,GO:0071248,BP,e,cellular response to metal ion,26/530,66/4382,1.187717e-08,5,0.000135,4.408171e-07,"142, 207, 468, 891, 999, 1027, 1050, 1147, 138...",33641663,These findings suggest that TMEM52B is a novel...
5,GO:0071248,BP,e,cellular response to metal ion,26/530,66/4382,1.187717e-08,5,0.000135,4.408171e-07,"142, 207, 468, 891, 999, 1027, 1050, 1147, 138...",33761370,The epithelial to mesenchymal transition (EMT)...


In [123]:
goa_pubmed_bp_df.sort_values(by="p_fdr_bh").head(100)["depth"].mean()

5.99

In [120]:
goa_pubmed_bp_df["depth"].mean()

6.463740458015267

In [111]:
goa_pubmed_bp_df.to_csv("datasets/pln_goa_comp/goa_pubmed_bp.csv")